In [2]:
import cv2 as cv
import numpy as np
import onnxruntime as ort
import time 

path_to_frozen_inference_graph = 'frozen_inference_graph.pb'
path_coco_model= 'mask_rcnn_inception_v2_coco_2018_01_28.pbtxt'
labels = ['convertible', 'hatchback', 'sedan', 'sport', 'suv', 'truck', 'van']

cap = cv.VideoCapture('test5.mp4')
_, frame = cap.read()
height, width, _ = frame.shape

session = ort.InferenceSession('model.onnx')

input_name = session.get_inputs()[0].name
input_shape = session.get_inputs()[0].shape
input_type = session.get_inputs()[0].type

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
        
    colors = np.random.randint(125, 255, (80, 3))
    img = cv.resize(frame,(width,height))
    blob = cv.dnn.blobFromImage(img, swapRB=True)
    time2 = time.time()

    net = cv.dnn.readNetFromTensorflow(path_to_frozen_inference_graph,path_coco_model)
    net.setInput(blob)

    boxes, _ = net.forward(["detection_out_final", "detection_masks"])

    detection_count = boxes.shape[2]

    for i in range(detection_count):
        box = boxes[0, 0, i]
        class_id = box[1]
        score = box[2]
        if score < 0.7:
             continue
        if class_id == 2:
            x = int(box[3] * width)
            y = int(box[4] * height)
            x2 = int(box[5] * width)
            y2 = int(box[6] * height)
            cv.rectangle(img, (x, y), (x2, y2), (255, 0, 0), 3)    
            
            to_resize = img[y:y2, x:x2]
            resized = cv.resize(to_resize, (300, 300))
            input_tensor = np.transpose(resized, (2, 0, 1))
            input_tensor = np.expand_dims(input_tensor, axis=0)
            input_tensor = input_tensor.astype(np.float32) 
            
            output = session.run(None, {input_name: input_tensor})
            print(f'output: ${output}')
            index = np.argmax(output)
            cv.rectangle(img, (x, y), (x+100, y-30), (155, 155, 155), -1)
            cv.putText(img, labels[index], (x+10, y-10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)

    cv.imshow('Frame', img)
    
    if cv.waitKey(1) == ord('q'):
        break


cap.release()
cv.destroyAllWindows()

time2-1: $0.0053539276123046875
time3-2: $0.5906291007995605
time4-3: $0.0011739730834960938
time5-4: $1.7319560050964355
time6-5: $0.00013899803161621094
output: $[array([[0.00253065, 0.15729286, 0.29530334, 0.00094172, 0.34438804,
        0.01779511, 0.18174827]], dtype=float32)]
output: $[array([[1.4692421e-04, 2.5093928e-02, 1.1734124e-02, 1.9286650e-04,
        5.4551058e-02, 6.1270143e-03, 9.0215415e-01]], dtype=float32)]
output: $[array([[0.00137263, 0.27241075, 0.02399489, 0.00535194, 0.05876473,
        0.00394828, 0.6341568 ]], dtype=float32)]
output: $[array([[0.07489336, 0.08870813, 0.1613093 , 0.34127837, 0.14960487,
        0.08521093, 0.09899499]], dtype=float32)]
output: $[array([[0.01409232, 0.04659037, 0.0087075 , 0.00588632, 0.16232729,
        0.06468326, 0.69771296]], dtype=float32)]
output: $[array([[0.08314563, 0.11938524, 0.08717025, 0.19560695, 0.31260082,
        0.02983323, 0.17225784]], dtype=float32)]
output: $[array([[0.12366568, 0.11514975, 0.10868783, 0.

KeyboardInterrupt: 